In [1]:
import warnings
warnings.filterwarnings("ignore")
from threeML import *




Bad key "text.kerning_factor" on line 4 in
/data/disk01/home/jasonfan/miniconda3/envs/p3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution

WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Exp

In [2]:
from hawc_hal import HAL, HealpixConeROI
import matplotlib.pyplot as plt


# Define the ROI
ra_crab, dec_crab = 83.63,22.02
data_radius = 3.0
model_radius = 8.0

roi = HealpixConeROI(data_radius=data_radius,
                     model_radius=model_radius,
                     ra=ra_crab,
                     dec=dec_crab)

# Instance the plugin


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
maptree = "../data/HAWC_9bin_507days_crab_data.hd5"
response = "../data/HAWC_9bin_507days_crab_response.hd5"

hawc = HAL("HAWC",
           maptree,
           response,
           roi,
           flat_sky_pixels_size=0.1)

# Use from bin 1 to bin 9
hawc.set_active_measurements(1, 9)

# Display information about the data loaded and the ROI
hawc.display()

# Look at the data
fig = hawc.display_stacked_image(smoothing_kernel_sigma=0.17)

# Save to file
fig.savefig("public_crab_logParabola_stacked_image.png")


# Define model
spectrum = Log_parabola()
source = PointSource("crab", ra=ra_crab, dec=dec_crab, spectral_shape=spectrum)

spectrum.piv = 7 * u.TeV
spectrum.piv.fix = True

spectrum.K = 1e-14 / (u.TeV * u.cm ** 2 * u.s)  # norm (in 1/(keV cm2 s))
spectrum.K.bounds = (1e-35, 1e-10) / (u.TeV * u.cm ** 2 * u.s)  # without units energies are in keV


spectrum.alpha = -2.5  # log parabolic alpha (index)
spectrum.alpha.bounds = (-4., 2.)

spectrum.beta = 0  # log parabolic alpha (index)
spectrum.beta.bounds = (-4., 2.)

model = Model(source)

data = DataList(hawc)

jl = JointLikelihood(model, data, verbose=False)
jl.set_minimizer("minuit")
param_df, like_df = jl.fit()

results=jl.results
results.write_to("crab_lp_public_results.fits", overwrite=True)
results.optimized_model.save("crab_fit.yml", overwrite=True)

# See the model in counts space and the residuals
fig = hawc.display_spectrum()

# Save it to file
fig.savefig("public_crab_logParabola_residuals.png")

# See the spectrum fit
fig = plot_spectra(jl.results,
                   ene_min=1.0,
                   ene_max=37,
                   num_ene=50,
                   energy_unit='TeV',
                   flux_unit='TeV/(s cm2)')
plt.xlim(0.8,100)
plt.ylabel(r"$E^2\,dN/dE$ [TeV cm$^{-2}$ s$^{-1}$]")
plt.xlabel("Energy [TeV]")
fig.savefig("public_crab_fit_spectrum.png")

# Look at the different energy planes (the columns are model, data, residuals)
fig = hawc.display_fit(smoothing_kernel_sigma=0.3,display_colorbar=True)
fig.savefig("public_crab_fit_planes.png")

# Compute TS
TS = jl.compute_TS("crab", like_df)
print(TS)



WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



Creating singleton for /data/disk01/home/jasonfan/umd_icecube_analysis_tutorial/test3ml/workshop/data/HAWC_9bin_507days_crab_response.hd5
Region of Interest: 
-------------------

HealpixConeROI: Center (R.A., Dec) = (83.630, 22.020), data radius = 3.000 deg, model radius: 8.000 deg

Flat sky projection: 
--------------------

Width x height: 160 x 160 px
Pixel sizes: 0.1 deg

Response: 
---------

Response file: /data/disk01/home/jasonfan/umd_icecube_analysis_tutorial/test3ml/workshop/data/HAWC_9bin_507days_crab_response.hd5
Number of dec bins: 101
Number of energy/nHit planes per dec bin_name: 9

Map Tree: 
----------



,Bin,Nside,Scheme,Obs counts,Bkg counts,obs/bkg,Pixels in ROI,Area (deg^2)
0,0,1024,RING,450572866.0,4.504941e+08,1.000175,8629,28.290097
1,1,1024,RING,31550239.0,3.150819e+07,1.001335,8629,28.290097
2,2,1024,RING,10080669.0,1.005106e+07,1.002945,8629,28.290097
3,3,1024,RING,2737051.0,2.721557e+06,1.005693,8629,28.290097
4,4,1024,RING,346666.0,3.400225e+05,1.019538,8629,28.290097
5,5,1024,RING,77507.0,7.437867e+04,1.042060,8629,28.290097
6,6,1024,RING,14141.0,1.321418e+04,1.070139,8629,28.290097
7,7,1024,RING,8279.0,7.735450e+03,1.070267,8629,28.290097
8,8,1024,RING,2240.0,2.047489e+03,1.094023,8629,28.290097
9,9,1024,RING,3189.0,2.950479e+03,1.080842,8629,28.290097


This Map Tree contains 508.195 transits in the first bin
Total data size: 1.38 Mb

Active energy/nHit planes (9):
-------------------------------

['1', '2', '3', '4', '5', '6', '7', '8', '9']



WARNING RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



...

,-log(likelihood)
HAWC,18765.836792
total,18765.836792



Values of statistical measures:



,statistical measures
AIC,37537.673861
BIC,37565.769990


Propagating errors :

[***********100%*************]  50 / 50  in 5.6 s (0:00:00 remaining)
Smoothing maps :

[***********100%*************]  9 / 9  in 3.0 s (0:00:00 remaining)
       Null hyp.     Alt. hyp.            TS
HAWC  24433.3938  18765.836792  11335.114017



WARNING RuntimeWarning: External parameter HAWC_bkg_renorm already exist in the model. Overwriting it...


WARNING RuntimeWarning: There is no free parameter in the current model


WARNING RuntimeWarning: External parameter HAWC_bkg_renorm already exist in the model. Overwriting it...

